# Benchmark Analysis

### DOI histogram ground truth

In [ ]:
import pandas as pd
import altair as alt
from notebook_test_case import *
from analysis import *

ground_truth_df = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df, PARAMETERS.n_bins)
ground_truth_bins.columns = ["count"]
ground_truth_bins["bin"] = ground_truth_bins.index

alt.Chart(ground_truth_bins).mark_bar().encode(
  x=alt.X("bin:N"),
  y=alt.Y("count:Q"),
  tooltip="count:Q",
).properties(
  width=400,
  height=100,
)

### Sample of the dataset under analysis

In [ ]:
import pandas as pd
import altair as alt
from notebook_test_case import *

df = pd.read_csv(DATA.data_path)
doi = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")
df_ = df.copy()
df_["doi"] = 0
sample = df_[:10000]

print(df.shape, doi.shape)

points = alt.Chart(sample).mark_circle().encode(
  x=alt.X(field=DATA.numeric_columns[0], type="quantitative"),
  y=alt.Y(field=DATA.numeric_columns[-1], type="quantitative"),
  # color=alt.Color("doi:Q")
  opacity={"value": 0.1}
)
grid = alt.Chart(sample).mark_bar().encode(
  x=alt.X(field=DATA.numeric_columns[0], type="quantitative", bin=True),
  y=alt.Y(field=DATA.numeric_columns[-1], type="quantitative", bin=True),
  color=alt.Color("count():Q", scale=alt.Scale(scheme="blues"))
)
grid + points

### Bigger chunks vs. ground truth

In [ ]:
import altair as alt
from notebook_test_case import *


ground_truth_df = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df, PARAMETERS.n_bins, with_labels=True)

bigger_chunks_df = pd.read_csv(f"{PATH}/doi/__bigger_chunks__.csv")
bigger_chunks_bins = get_doi_bins_df(bigger_chunks_df, PARAMETERS.n_bins, with_labels=True)

delta_bins_df = get_doi_delta_bins_df(bigger_chunks_bins[0], ground_truth_bins[0], PARAMETERS.total_size, 
                                      PARAMETERS.n_bins)

alt.Chart(delta_bins_df).mark_bar(size=5).encode(
  x=alt.X("bin:Q"),
  y=alt.Y("delta:Q"),
  tooltip=alt.Tooltip(["bin", "delta"]),
).properties(
  width=100,
  height=100
)

## Strategy-based Analysis

### DOI distributions per use case in histograms

In [ ]:
import os
import altair as alt
from notebook_test_case import PATH, PARAMETERS, CONTEXT_STRATEGIES, UPDATE_STRATEGIES
from analysis import *

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{PATH}/doi")

# compute the ground truth bins 
ground_truth_df = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df, PARAMETERS.n_bins)

all_doi_bins_df = pd.DataFrame()

# compute the bins for each combination of strategies and then compare it to the ground truth in a 
# layered histogram
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # bin the doi data 
    df = pd.read_csv(f"{PATH}/doi/{test_case}")
    df_bins = get_doi_bins_df(df, PARAMETERS.n_bins)

    df_bins.columns = ["count"]
    df_bins["bin"] = df_bins.index
    df_bins["context_strategy"] = c_strat[0]
    df_bins["update_strategy"] = u_strat[0]

    all_doi_bins_df = all_doi_bins_df.append(df_bins)

all_doi_bins_df.reset_index(inplace=True, drop=True)

print(all_doi_bins_df)

alt.Chart(all_doi_bins_df).mark_bar().encode(
  x=alt.X("bin:N"),
  y=alt.Y("count:Q"),
  tooltip=alt.Tooltip(["bin", "count"]),
).properties(
  width=100,
  height=100
).facet(
  row="context_strategy",
  column="update_strategy",
  spacing=10
)

### Difference in DOI distributions per use case in histograms

In [ ]:
import os
import altair as alt
from notebook_test_case import PATH, PARAMETERS, CONTEXT_STRATEGIES, UPDATE_STRATEGIES
from analysis import *

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{PATH}/doi")

# compute the ground truth bins 
ground_truth_df = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df, PARAMETERS.n_bins)

all_doi_bins_df = pd.DataFrame()

# compute the bins for each combination of strategies and then compare it to the ground truth in a 
# layered histogram
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # bin the doi data 
    df = pd.read_csv(f"{PATH}/doi/{test_case}")
    bins_df = get_doi_bins_df(df, PARAMETERS.n_bins)

    doi_delta_bins = get_doi_delta_bins_df(bins_df, ground_truth_bins)

    doi_delta_bins["context_strategy"] = c_strat[0]
    doi_delta_bins["update_strategy"] = u_strat[0]

    all_doi_bins_df = all_doi_bins_df.append(doi_delta_bins)

max_count = ground_truth_bins.max()[0]

alt.Chart(all_doi_bins_df).mark_bar().encode(
  x=alt.X("bin:Q"),
  y=alt.Y("delta:Q", scale=alt.Scale(domain=[-max_count, max_count])),
  tooltip=alt.Tooltip(["bin", "delta"]),
).properties(
  width=100,
  height=100
).facet(
  row="context_strategy",
  column="update_strategy",
  spacing=10
)

### Bin-based accuracy per item per test case

In [ ]:
from notebook_test_case import *
from analysis import *
import pandas as pd
import altair as alt
from database import ID, DOI
from test_case import get_full_title


id = ID.lower()
doi = DOI.lower()
gt = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")
gt_bin_labels = get_doi_bins_df(gt, PARAMETERS.n_bins, with_labels=True)[1]

available_test_cases = os.listdir(f"{PATH}/doi")
results = pd.DataFrame()

# compute the overlap between the two 
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # read the benchmark results
    df = pd.read_csv(f"{PATH}/doi/{test_case}")
    bin_labels = get_doi_bins_df(df, PARAMETERS.n_bins, True)[1]
    diff = get_doi_bin_error_df(gt_bin_labels, bin_labels)
    diff["context_strategy"] = c_strat[0]
    diff["update_strategy"] = u_strat[0]
    
    results = results.append(diff)

strategies_chart = alt.Chart(results).mark_bar().encode(
  x="count:Q",
  y=alt.Y("update_strategy:N", title=None),
  row="context_strategy:N",
  stroke={"value": "#fff"},
  color=alt.Color("diff:Q", scale=alt.Scale(scheme='viridis')),
  tooltip=["diff", "count"],
)

bigger_chunks_df = pd.read_csv(f"{PATH}/doi/__bigger_chunks__.csv")
bigger_chunks_bins = get_doi_bins_df(bigger_chunks_df, PARAMETERS.n_bins, with_labels=True)

doi_bin_error_df = get_doi_bin_error_df(bigger_chunks_bins[1], gt_bin_labels)

bigger_chunks_chart = alt.Chart(doi_bin_error_df).mark_bar().encode(
  x="count:Q",
  stroke={"value": "#fff"},
  color=alt.Color("diff:Q", scale=alt.Scale(scheme='viridis')),
  tooltip=["diff", "count"],
).properties(
  title="Bigger chunks:"
)

print("doi accuracy per item:")
alt.vconcat(strategies_chart, bigger_chunks_chart).properties(
  title=get_full_title(DOI_CONFIG, PARAMETERS, DATA),
)

### Error per item per strategy

In [ ]:
import os
from notebook_test_case import *
from test_case import get_full_title
from analysis import *
import pandas as pd
import altair as alt


gt = pd.read_csv(f"{PATH}/doi/__ground_truth__.csv")

available_test_cases = os.listdir(f"{PATH}/doi")
strategies_error = pd.DataFrame()

# compute the overlap between the two 
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # read the benchmark results
    df = pd.read_csv(f"{PATH}/doi/{test_case}")
    strategy_error = get_doi_error_df(gt, df, absolute=False)

    strategy_error["context_strategy"] = c_strat[0]
    strategy_error["update_strategy"] = u_strat[0]
    strategies_error = strategies_error.append(strategy_error)

bigger_chunks_df = pd.read_csv(f"{PATH}/doi/__bigger_chunks__.csv")
bigger_chunks_error = get_doi_error_df(gt, bigger_chunks_df, absolute=False)

doi_max = max(strategies_error["doi"].max(), bigger_chunks_error["doi"].max())

strategy_chart = alt.Chart(strategies_error).mark_boxplot().encode(
  x=alt.X("doi:Q", title=None, scale=alt.Scale(domain=[-doi_max, doi_max])),
  y=alt.Y("context_strategy:N", title=None),
  opacity=alt.condition(
    (alt.datum.context_strategy == "no context") |  (alt.datum.update_strategy == "no update"), 
    alt.value(0.3),
    alt.value(1)
  ),
  row="update_strategy:N",
  tooltip="doi:Q"
)

bigger_chunks_chart = alt.Chart(bigger_chunks_error).mark_boxplot().encode(
  x=alt.X("doi:Q", title=None, scale=alt.Scale(domain=[-doi_max, doi_max])),
  tooltip="doi:Q"
).properties(
  title="using bigger chunks:"
)

alt.vconcat(strategy_chart, bigger_chunks_chart).properties(
  title=get_full_title(DOI_CONFIG, PARAMETERS, DATA),
)

### Error difference sequential vs. mixed-in context computation

In [ ]:
from notebook_test_case import *
from analysis import *
import pandas as pd
import altair as alt

available_test_cases = os.listdir(f"{PATH}/doi")
strategies_diff = pd.DataFrame()

# compute the overlap between the two 
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # read the benchmark results
    df_mixed = pd.read_csv(f"{PATH}/doi/{test_case}")
    df_sequential = pd.read_csv(f"{PATH}/doi-seq/{test_case}")
    
    strategy_diff = get_doi_error_df(df_sequential, df_mixed, absolute=False)
    strategy_diff["context_strategy"] = c_strat[0]
    strategy_diff["update_strategy"] = u_strat[0]
    
    strategies_diff = strategies_diff.append(strategy_diff)

grouped = strategies_diff.groupby(["context_strategy", "update_strategy"]).mean()
grouped = pd.DataFrame(grouped).reset_index()

alt.Chart(grouped).mark_bar().encode(
  x=alt.X("context_strategy:N", title=None),
  y=alt.Y("doi:Q", title=None, scale=alt.Scale(domain=[-0.25, 0.25])),
  opacity=alt.condition(
    (alt.datum.context_strategy == "no context") |  (alt.datum.update_strategy == "no update"), 
    alt.value(0.3),
    alt.value(1)
  ),
  color=alt.condition(
    alt.datum.value > 0,
    alt.value("firebrick"),
    alt.value("GREEN"),
  ),
  column=alt.Column("update_strategy:N"),
  tooltip="doi:Q"
).properties(
  height=100,
  width=175,
)

### Total time per test case in boxplots

In [ ]:
import altair as alt
from notebook_test_case import *

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{PATH}/times")
available_test_cases

all_doi_values_df = pd.DataFrame()

# build one big dataframe containing all doi scores and label each based on the strategies that were
# used to generate them
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{PATH}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_doi_values_df = all_doi_values_df.append(df)
    all_doi_values_df.reset_index(inplace=True, drop=True)

chart1 = alt.Chart(all_doi_values_df).mark_boxplot().encode(
  x="update_strategy:N",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "linear"}, "title": "time (s)"},
  column="context_strategy:N",
  color=alt.value("#BEAED4")
).properties(
  width=120,
  height=250
)

chart2 = alt.Chart(all_doi_values_df).mark_boxplot().encode(
  x="context_strategy:N",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "linear"}, "title": "time (s)"},
  column="update_strategy:N",
  color=alt.value("#FDC086")
).properties(
  width=120,
  height=250,
)

alt.vconcat(chart1, chart2).properties(
  title=get_full_title(DOI_CONFIG, PARAMETERS, DATA),
)

### Aggregated total time per strategy in boxplots

In [ ]:
import altair as alt

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{PATH}/times")
available_test_cases

all_time_series_df = pd.DataFrame()

# build one big dataframe containing all doi scores and label each based on the strategies that were
# used to generate them
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{PATH}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_time_series_df = all_time_series_df.append(df)
    all_time_series_df.reset_index(inplace=True, drop=True)

chart1 = alt.Chart(all_time_series_df).mark_boxplot().encode(
  x="context_strategy:N",
  y=alt.Y("total_time:Q", title="time (s)"),
  color=alt.value("#BEAED4")
).properties(
  width=320,
  height=150
)

chart2 = alt.Chart(all_time_series_df).mark_boxplot().encode(
  x="update_strategy:N",
  y=alt.Y("total_time:Q", title="time (s)"),
  color=alt.value("#FDC086")
).properties(
  width=320,
  height=150
)

alt.hconcat(chart1, chart2).properties(
  title=get_full_title(DOI_CONFIG, PARAMETERS, DATA),
)

### Time series for each test case step

In [ ]:
import altair as alt
import pandas as pd

all_timeseries_df = pd.DataFrame()

for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{PATH}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_timeseries_df = all_timeseries_df.append(df)
    all_timeseries_df.reset_index(inplace=True, drop=True)

# temporal_columns = ["chunk_time", "context_time", "outdated_time", "old_doi_time", 
#    "store_new_time", "update_dois_time", "total_time"]

temporal_columns = ["chunk_time", "context_time", "outdated_time"]

alt.Chart(all_timeseries_df).transform_fold(
  temporal_columns
).mark_line().encode(
  x="chunk:N",
  y="value:Q",
  color="key:N",
  column="update_strategy:N",
  row="context_strategy:N",
  tooltip="temporal_columns:Q"
).properties(
  height=100,
  width=200,
  spacing=0,
)

### Aggregated Time series per strategy

In [ ]:
import altair as alt

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{PATH}/times")
available_test_cases

all_timeseries_df = pd.DataFrame()

# build one big dataframe containing all doi scores and label each based on the strategies that were
# used to generate them
for c_strat in CONTEXT_STRATEGIES:
  for u_strat in UPDATE_STRATEGIES:
    # check if that test case exists
    test_case = f"{u_strat[0]}-{c_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{PATH}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_timeseries_df = all_timeseries_df.append(df)
    all_timeseries_df.reset_index(inplace=True, drop=True)

alt.data_transformers.disable_max_rows()
alt.Chart(all_timeseries_df).mark_line().encode(
  x="chunk:Q",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "linear"}, "title": "time (s)"},
  row="context_strategy:N",
  color="update_strategy:N",
).properties(
  width=800,
  height=120
)

## Analyze test case

### Select test case

In [ ]:
import os
from os.path import join
from ipywidgets import widgets
from IPython.display import display

out_folder = "./out/"
test_cases = [f for f in os.listdir(out_folder) if os.path.isfile(join(out_folder, f))]
test_cases = [tc for tc in test_cases if "strategies" not in tc]

if len(test_cases) == 0:
  print("no test cases were found that are not strategy-based.")

doi_test_case_dropdown = widgets.Dropdown(
  description="Non-strategy runs",
  options=test_cases,
  disabled=len(test_cases) == 0,
  style={"description_width": "initial"},
)
display(doi_test_case_dropdown)

### Error per item

In [ ]:
import os
import json
from os.path import join
from notebook_test_case import PATH, TEST_CASE_INDEX
from benchmarks import load_test_case, MODES
from test_case import get_full_title
from analysis import get_strategy_bc_errors
import pandas as pd
import altair as alt


if doi_test_case_dropdown.value is None:
  raise Exception("select a test case first using the widget above")

selected_test_case = json.load(open("./out/"+doi_test_case_dropdown.value))

test_case_doi_paths = [t["dois_path"] for t in selected_test_case["test_cases"]]
test_case_preset = load_test_case(selected_test_case["preset_index"])
doi_file_name = f"{test_case_preset.name}.csv"  # filename consists of the strategies in the preset
data_label = test_case_preset.data.name
params_label = test_case_preset.params.name

mode = selected_test_case["mode"]
print(MODES, mode, MODES.index(mode))

strategies_error, bigger_chunks_errors = get_strategy_bc_errors(
  path_list=test_case_doi_paths,
  file_name=doi_file_name,
  label=mode,
  absolute=False
)

doi_max = max(strategies_error["doi"].max(), bigger_chunks_errors["doi"].max())

strategy_chart = alt.Chart(strategies_error).mark_boxplot().encode(
  x=alt.X("doi:Q", title=None, scale=alt.Scale(domain=[-doi_max, doi_max])),
  y=alt.Y(f"{mode}:N", title=None),
  # row="update_strategy:N",
  tooltip="doi:Q"
).properties(
  title=f"{doi_file_name}",
)

bigger_chunks_chart = alt.Chart(bigger_chunks_errors).mark_boxplot().encode(
  x=alt.X("doi:Q", title=None, scale=alt.Scale(domain=[-doi_max, doi_max])),
  y=alt.Y(f"{mode}:N", title=None),
  # row="update_strategy:N",
  tooltip="doi:Q"
).properties(
  title="using bigger chunks:"
)

alt.vconcat(strategy_chart, bigger_chunks_chart).properties(
  title=f"Prediction error for {data_label} at {params_label}",
)

### Bin-based difference

In [ ]:
import os
import altair as alt
from notebook_test_case import PATH, PARAMETERS, CONTEXT_STRATEGIES, UPDATE_STRATEGIES
from analysis import *

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"./out/taxis-shuffled/outlierness/1200000/1000/doi")

# compute the ground truth bins 
ground_truth_df = pd.read_csv(f"./out/taxis-shuffled/outlierness/12000/1000/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df, PARAMETERS.n_bins)


if doi_test_case_dropdown.value is None:
  raise Exception("select a test case first using the widget above")

selected_test_case = json.load(open("./out/"+doi_test_case_dropdown.value))

test_case_doi_paths = [t["dois_path"] for t in selected_test_case["test_cases"]]
test_case_preset = load_test_case(selected_test_case["preset_index"])
doi_file_name = f"{test_case_preset.name}.csv"  # filename consists of the strategies in the preset
data_label = test_case_preset.data.name
params_label = test_case_preset.params.name

mode = selected_test_case["mode"]
for test_case_path in test_case_doi_paths:
  # bin the doi data 
  df = pd.read_csv(f"{test_case_path}/{doi_file_name}")
  df_bins = get_doi_bins_df(df, PARAMETERS.n_bins)

  df_bins.columns = ["count"]
  df_bins["bin"] = df_bins.index

  if mode == "dois":
    test_case_label = test_case_path.split("/")[3]
  elif mode == "datasets":
    test_case_label = test_case_path.split("/")[2]
  elif mode == "parameters":
    test_case_label = "".join(test_case_path.split("/")[-3:-1])  # include data and chunk sizes

  df_bins["mode"] = test_case_label

  all_doi_bins_df = all_doi_bins_df.append(df_bins)

all_doi_bins_df.reset_index(inplace=True, drop=True)

alt.Chart(all_doi_bins_df).mark_bar().encode(
  x=alt.X("bin:N"),
  y=alt.Y("count:Q"),
  tooltip=alt.Tooltip(["bin", "count"]),
).properties(
  width=100,
  height=100
).facet(
  column="mode",
  spacing=10
)

### Analyze Composite Test Case

In [ ]:
import os
from os.path import join
from ipywidgets import widgets, HBox
from IPython.display import display

out_folder = "./out/"
test_cases = [f for f in os.listdir(out_folder) if os.path.isfile(join(out_folder, f))]
test_cases = [tc for tc in test_cases if "composite" in tc]

if len(test_cases) == 0:
  print("no test cases found.")

composite_test_case_dropdown = widgets.Dropdown(
  description="test case:",
  options=test_cases,
  disabled=len(test_cases) == 0,
  style={"description_width": "initial"},
)

mode_dropdown = widgets.Dropdown(
  description="show:",
  options=["datasets", "strategies", "dois", "parameters"],
)

facet_dropdown = widgets.Dropdown(
  description="facet by",
  options=["context_strategy", "update_strategy", "storage_strategy"],
)


HBox([composite_test_case_dropdown, mode_dropdown, facet_dropdown])

### Analyze by mode

In [ ]:
import os
import json
from os.path import join
import notebook_test_case  # for unsetting the altair row limit
import numpy as np
import pandas as pd
import altair as alt

from benchmarks import load_test_case
from test_case import get_full_title
from analysis import get_strategy_bc_errors


if composite_test_case_dropdown.value is None or mode_dropdown.value is None:
  raise Exception("select a test case and mode with the widget above")

selected_test_case = json.load(open("./out/"+composite_test_case_dropdown.value))
test_cases = selected_test_case["test_cases"]
test_case_doi_paths = [tc["dois_path"] for tc in test_cases]

context_strategies = np.unique([tc["context_strategy"] for tc in test_cases]).tolist()
update_strategies = np.unique([tc["update_strategy"] for tc in test_cases]).tolist()
storage_strategies = np.unique([tc["storage_strategy"] for tc in test_cases]).tolist()

mode = mode_dropdown.value

all_strategies_error = pd.DataFrame()
all_bigger_chunks_error = pd.DataFrame()

for c in context_strategies:
  for u in update_strategies:
    for s in storage_strategies:
      file_name = f"{c}-{u}-{s}.csv"

      strategies_error, bigger_chunks_error = get_strategy_bc_errors(
        path_list=test_case_doi_paths,
        file_name=file_name,
        label=mode,
        absolute=False
      )
      strategies_error["context_strategy"] = c
      strategies_error["update_strategy"] = u
      strategies_error["storage_strategy"] = s

      bigger_chunks_error["context_strategy"] = c
      bigger_chunks_error["update_strategy"] = u
      bigger_chunks_error["storage_strategy"] = s

      all_strategies_error = all_strategies_error.append(strategies_error)
      all_bigger_chunks_error = all_bigger_chunks_error.append(bigger_chunks_error)

      # mean_strategy_error = strategies_error.groupby([mode]).mean().reset_index()
      # mean_bc_error = bigger_chunks_errors.groupby([mode]).mean().reset_index()
      # strategy_errors = strategy_errors.append(mean_strategy_error)
      # bigger_chunk_errors = bigger_chunk_errors.append(mean_bc_error)

print(all_strategies_error)
doi_max = max(all_strategies_error["doi"].max(), all_bigger_chunks_error["doi"].max())

strategy_chart = alt.Chart(all_strategies_error).mark_boxplot().encode(
  x=alt.X("doi:Q", title=None, scale=alt.Scale(domain=[-doi_max, doi_max])),
  y=alt.Y(f"{mode}:N", title=None),
  row=f"{facet_dropdown.value}:N",
  tooltip="doi:Q"
).properties(
  title=f"{composite_test_case_dropdown.value}",
)

bigger_chunks_chart = alt.Chart(all_bigger_chunks_error).mark_boxplot().encode(
  x=alt.X("doi:Q", title=None, scale=alt.Scale(domain=[-doi_max, doi_max])),
  y=alt.Y(f"{mode}:N", title=None),
  row=f"{facet_dropdown.value}:N",
  tooltip="doi:Q"
).properties(
  title="using bigger chunks:"
)

alt.vconcat(strategy_chart, bigger_chunks_chart)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

# Load the example dataset of brain network correlations
df = sns.load_dataset("brain_networks", header=[0, 1, 2], index_col=0)

# Pull out a specific subset of networks
used_networks = [1, 3, 4, 5, 6, 7, 8, 11, 12, 13, 16, 17]
used_columns = (df.columns.get_level_values("network")
                          .astype(int)
                          .isin(used_networks))
df = df.loc[:, used_columns]

# Compute the correlation matrix and average over networks
corr_df = df.corr().groupby(level="network").mean()
corr_df.index = corr_df.index.astype(int)
corr_df = corr_df.sort_index().T

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 6))

print(corr_df)
# Draw a violinplot with a narrower bandwidth than the default
sns.violinplot(data=corr_df, palette="Set3", bw=.2, cut=1, linewidth=1)

# Finalize the figure
ax.set(ylim=(-.7, 1.05))
sns.despine(left=True, bottom=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

import pandas as pd
import numpy as np


# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 6))

df = all_strategies_error
df["bc"] = all_bigger_chunks_error["doi"].to_numpy()

df_a = df[["doi", "datasets", "context_strategy", "update_strategy", "storage_strategy"]]
df_b = df[["bc", "datasets", "context_strategy", "update_strategy", "storage_strategy"]]
df_b.columns = ["doi", "datasets", "context_strategy", "update_strategy", "storage_strategy"]
df_a["class"] = "strategy"
df_b["class"] = "bigger_chunks"
df = df_a.append(df_b)
df.reset_index(inplace=True)

df = df.set_index(["context_strategy", "update_strategy", "storage_strategy"])

# Draw a violinplot with a narrower bandwidth than the default
sns.violinplot(data=df, x="datasets", y="doi", palette="Set3", hue="class", bw=.2, cut=1, linewidth=1)

# Finalize the figure
ax.set(ylim=(-.7, 1.05))
sns.despine(left=True, bottom=True)